In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_LAUNCH_BLOCKING=1
import torch

from main import init_model, init_training, create_connections_datasets

env: CUDA_LAUNCH_BLOCKING=1


/Users/brianfitzgerald/Documents/GitHub/minRL/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer, model = init_model()

2025-05-04 19:48:36.360 | INFO     | main:init_model:42 - Model loaded.
2025-05-04 19:48:36.362 | INFO     | main:init_model:45 - Using device mps


In [3]:
train_dataset, _ = create_connections_datasets(tokenizer)

In [4]:
training_objects = init_training(model, train_dataset)

In [36]:
from main import init_training
from grpo import rollout, update_policy

from tasks.countdown import reward_function

to = init_training(model, train_dataset)
batch = next(iter(to.train_dataloader))

episodes = rollout(
    model=model,
    tokenizer=tokenizer,
    batch=batch,
    max_gen_len=to.config.max_gen_len,
    num_answer_per_question=to.config.num_answer_per_question,
    reward_function=reward_function,
    device=to.device,
)
if to.config.skip_unfinished_episodes:
    episodes = [episode for episode in episodes if episode.is_finished]

2025-05-04 19:58:56.285 | INFO     | grpo:rollout:43 - Generating responses for 1 prompts, max_gen_len=100
2025-05-04 19:59:02.159 | INFO     | grpo:rollout:73 - Generated token ids: 100
2025-05-04 19:59:02.159 | INFO     | grpo:rollout:80 - Generated token ids after removing padding: 100
2025-05-04 19:59:02.160 | INFO     | grpo:rollout:87 - Generated text: 

Okay, let's see. The user provided a list of words: file, history, view, window, asteroids, breakout, frogger, pong, dip, drop, fall, sink, advance, march, progress, push. I need to form four groups of four words each, with each word used only once. Let me start by looking for obvious categories.

First, "file," "view," "window," and "history" seem related to a digital environment.


rewards {'reward': 0.0, 'reward_info': {'format_reward': 0.0, 'answer_reward': 0.0}}


In [8]:
model(input.unsqueeze(0))

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 5.6875,  4.7188,  3.5781,  ...,  1.3750,  1.3750,  1.3750],
         [10.5000,  0.7227,  0.2070,  ..., -1.7109, -1.7109, -1.7109],
         [15.3125,  7.5938,  5.4375,  ..., -1.8984, -1.8984, -1.8984],
         [ 6.3125,  3.8125,  4.1562,  ...,  0.0510,  0.0510,  0.0510]]],
       device='mps:0', dtype=torch.bfloat16, grad_fn=<LinearBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x147a3cb60>, hidden_states=None, attentions=None)

In [35]:
# Update policy - compute loss and perform backward pass
results = update_policy(
    model=model,
    optimizer=to.optimizer,
    episodes=episodes,
    micro_batch_size=to.config.micro_batch_size,
    pad_token_id=tokenizer.pad_token_id,
    max_grad_norm=to.config.max_grad_norm,
    device=to.device,
    dtype=to.dtype,
    apply_loss=True
)


2025-05-04 19:58:49.470 | INFO     | grpo:update_policy:8 - Updating policy with 1 episodes, 100 target tokens, 1 micro-batches
* Computing policy gradient:  0/ 1mINFO     | grpo:update_policy:11 - 


episodes [0.0]
rewards tensor([0.], device='mps:0')
mean tensor(0., device='mps:0')
std 0
batch ep [0.0]


In [13]:
results